In [2]:
import pandas as pd
import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import itertools
import statistics

from sklearn.feature_selection import RFE
from sklearn.svm import SVR

In [5]:
inp = sorted(
    glob.glob('../data/r7/*.tsv'))

In [6]:
inp

['../data/r7/colon-adeno_transcriptomics_cell-line+CPTAC.tsv',
 '../data/r7/colon-nos-adeno_transcriptomics_HCMI+CPTAC.tsv',
 '../data/r7/glioblastoma_transcriptomics_cell-line+CPTAC.tsv',
 '../data/r7/lung-adeno_transcriptomics_cell-line+CPTAC.tsv',
 '../data/r7/pancreatic-adeno_transcriptomics_cell-line+CPTAC.tsv',
 '../data/r7/pancreatic-nos-ductal-ad_transcriptomics_HCMI+CPTAC.tsv',
 '../data/r7/renal-clear-cell_transcriptomics_cell-line+CPTAC.tsv']

In [7]:
mb = sorted(
    glob.glob('../results/r7/mbatch/*.tsv'))

In [8]:
mb

['../results/r7/mbatch/colon-adeno_transcriptomics_cell-line+CPTAC.tsv',
 '../results/r7/mbatch/colon-nos-adeno_transcriptomics_HCMI+CPTAC.tsv',
 '../results/r7/mbatch/glioblastoma_transcriptomics_cell-line+CPTAC.tsv',
 '../results/r7/mbatch/lung-adeno_transcriptomics_cell-line+CPTAC.tsv',
 '../results/r7/mbatch/pancreatic-adeno_transcriptomics_cell-line+CPTAC.tsv',
 '../results/r7/mbatch/pancreatic-nos-ductal-ad_transcriptomics_HCMI+CPTAC.tsv',
 '../results/r7/mbatch/renal-clear-cell_transcriptomics_cell-line+CPTAC.tsv']

In [9]:
vae = sorted(
    glob.glob('../results/r7/*.tsv'))

In [10]:
vae

['../results/r7/colon-adeno_transcriptomics_cell-line+CPTAC.latent_space.tsv',
 '../results/r7/colon-nos-adeno_transcriptomics_HCMI+CPTAC.latent_space.tsv',
 '../results/r7/glioblastoma_transcriptomics_cell-line+CPTAC.latent_space.tsv',
 '../results/r7/lung-adeno_transcriptomics_cell-line+CPTAC.latent_space.tsv',
 '../results/r7/pancreatic-adeno_transcriptomics_cell-line+CPTAC.latent_space.tsv',
 '../results/r7/pancreatic-nos-ductal-ad_transcriptomics_HCMI+CPTAC.latent_space.tsv',
 '../results/r7/renal-clear-cell_transcriptomics_cell-line+CPTAC.latent_space.tsv']

### RFE

In [11]:
for pth in inp:
    df = pd.read_csv(
        pth, sep = '\t', index_col = 0)
    X = df.iloc[:, 2:]
    break

In [12]:
df

,System,Cancer_type,entrz_tran_1,entrz_tran_2,entrz_tran_9,entrz_tran_10,entrz_tran_12,entrz_tran_13,entrz_tran_14,entrz_tran_15,...,entrz_tran_111064649,entrz_tran_111188157,entrz_tran_112441426,entrz_tran_112441434,entrz_tran_112694756,entrz_tran_113455421,entrz_tran_113523638,entrz_tran_114483833,entrz_tran_115253422,entrz_tran_116033993
improve_sample_id,,,,,,,,,,,,,,,,,,,,,
smpl_id_3,cell-line,colon-adeno,1.400538,0.321928,3.400538,0.613532,2.223423,1.687061,6.732133,0.084064,...,0.000000,0.042644,0.000000,1.956057,2.443607,3.598127,0.00,0.070389,1.655352,0.0
smpl_id_7,cell-line,colon-adeno,0.137504,0.201634,2.849999,0.344828,0.084064,3.547203,6.582707,0.454176,...,0.000000,0.000000,0.000000,1.974529,2.641546,2.272023,0.00,0.000000,2.182692,0.0
smpl_id_9,cell-line,colon-adeno,1.464668,0.250962,2.976364,0.201634,0.713696,1.985500,7.281513,0.000000,...,0.000000,0.028569,0.014355,2.513491,3.434295,4.596339,0.00,0.000000,2.319040,0.0
smpl_id_88,cell-line,colon-adeno,0.028569,0.000000,2.503349,1.948601,0.238787,2.748461,6.966938,0.028569,...,0.000000,0.000000,0.000000,2.066950,2.885574,3.938286,0.00,0.042644,1.275007,0.0
smpl_id_201,cell-line,colon-adeno,3.646163,0.333424,1.929791,0.238787,0.505891,0.070389,6.457627,0.356144,...,0.084064,0.000000,0.000000,1.970854,3.303050,3.232661,0.00,0.000000,1.163499,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
smpl_id_2100,cptac,colon-adeno,2.360000,13.470000,7.790000,6.660000,6.090000,3.080000,11.530000,0.000000,...,3.080000,0.000000,0.000000,6.810000,13.160000,9.890000,3.72,0.000000,8.320000,0.0
smpl_id_2101,cptac,colon-adeno,0.000000,12.740000,8.760000,8.040000,7.160000,1.090000,11.760000,0.000000,...,6.220000,1.340000,0.000000,6.660000,13.220000,9.550000,4.37,0.000000,8.730000,0.0
smpl_id_2102,cptac,colon-adeno,2.000000,13.990000,8.360000,6.900000,9.850000,8.060000,10.760000,0.810000,...,5.300000,0.000000,0.000000,5.820000,12.840000,9.560000,4.09,0.000000,6.940000,0.0


In [ ]:
estimator = SVR(kernel="linear")
rfe = RFE(estimator=estimator, n_features_to_select=10, step=0.5)
file_n = pd.read_csv(pth, sep = '\t', index_col = 0) # read validation split file
X = file_n.iloc[:, 1:]
y = file_n.iloc[:, 0]
rfe.fit(X, y)
mask = rfe.support_
vs_dict[vs] = json.dumps(list(X.columns[mask]))

In [ ]:
# clf = tree.DecisionTreeClassifier()
# clf = LogisticRegression(random_state=0).fit(X, y)